In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import random
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pywt import wavedec

In [6]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          print(os.path.join(dirpath2, f))

../Preprocessing/db1/data_mfccRMScqtdb1L3.csv
../Preprocessing/db1/data_mfccRMScqtdb1L9.csv
../Preprocessing/db1/data_mfccRMScqtdb1L2.csv
../Preprocessing/db1/data_mfccRMScqtdb1L5.csv
../Preprocessing/db1/data_mfccRMScqtdb1L8.csv
../Preprocessing/db1/data_mfccRMScqtdb1L7.csv
../Preprocessing/db1/data_mfccRMScqtdb1L6.csv
../Preprocessing/db1/data_mfccRMScqtdb1L1.csv
../Preprocessing/db1/data_mfccRMScqtdb1L10.csv
../Preprocessing/db1/data_mfccRMScqtdb1L4.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L5.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L10.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L8.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L4.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L1.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L9.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L6.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L7.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L2.csv
../Preprocessing/db7/data_mfccDWTcqtdb7L3.csv
../Preprocessing/db8/data_mfccDWTcqtdb8L3.csv
../Preprocessing/db8/data_mfccDW

In [7]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          model = tf.keras.Sequential([
              # tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2])),
              # tf.keras.layers.Dense(32, activation='relu'),
              # tf.keras.layers.BatchNormalization(),
              # tf.keras.layers.Dense(16, activation='relu'),
              # tf.keras.layers.Dense(5, activation='sigmoid')

              tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True),
              # tf.keras.layers.Flatten(),
              # tf.keras.layers.Dense(32, activation='relu'),
              # # tf.keras.layers.BatchNormalization(),
              # tf.keras.layers.BatchNormalization(),
              tf.keras.layers.Dense(32,activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.LSTM(64, return_sequences=False),
              tf.keras.layers.Dropout(0.2),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(32, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.Dense(16, activation='relu', activity_regularizer=tf.keras.regularizers.l2(1e-4)),
              tf.keras.layers.BatchNormalization(),

              tf.keras.layers.Dense(5, activation='softmax')
          ])

          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])

          history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/db1/data_mfccRMScqtdb1L3.csv
Epoch 1/10
50/50 [==============================] - 11s 151ms/step - loss: 1.6153 - acc: 0.1775 - val_loss: 1.6111 - val_acc: 0.2000
Epoch 2/10
50/50 [==============================] - 7s 137ms/step - loss: 1.6010 - acc: 0.2525 - val_loss: 1.6096 - val_acc: 0.2800
Epoch 3/10
50/50 [==============================] - 7s 142ms/step - loss: 1.5812 - acc: 0.3262 - val_loss: 1.6065 - val_acc: 0.3250
Epoch 4/10
50/50 [==============================] - 8s 156ms/step - loss: 1.5688 - acc: 0.3075 - val_loss: 1.6013 - val_acc: 0.2900
Epoch 5/10
50/50 [==============================] - 8s 156ms/step - loss: 1.5391 - acc: 0.3738 - val_loss: 1.5947 - val_acc: 0.3650
Epoch 6/10
50/50 [==============================] - 6s 121ms/step - loss: 1.5150 - acc: 0.3487 - val_loss: 1.5813 - val_acc: 0.3850
Epoch 7/10
50/50 [==============================] - 7s 143ms/step - loss: 1.4967 - acc: 0.3762 - val_loss: 1.5650 - val_acc: 0.4350
Epoch 8/10


KeyboardInterrupt: 